In [1]:
import os
import pandas as pd
import numpy as np
import time
import gc
import pickle

In [2]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'

In [3]:
df_distance_list = []
df_distance_list.append(pd.read_pickle(OUTPUT + '20190629_dist_bond_dir_train_temp0.pickle'))
df_distance_list.append(pd.read_pickle(OUTPUT + '20190629_dist_bond_dir_train_temp1.pickle'))
df_distance_list.append(pd.read_pickle(OUTPUT + '20190629_dist_bond_dir_train_temp2.pickle'))
df_distance_list.append(pd.read_pickle(OUTPUT + '20190629_dist_bond_dir_train_temp3.pickle'))

df_distance = pd.concat(df_distance_list, axis=0)


In [4]:
def merge_atom(df, df_distance):
    df_merge_0 = pd.merge(df, df_distance, left_on=['molecule_name', 'atom_index_0'], right_on=['molecule_name', 'atom_index'])
    df_merge_0_1 = pd.merge(df_merge_0, df_distance, left_on=['molecule_name', 'atom_index_1'], right_on=['molecule_name', 'atom_index'])
    del df_merge_0_1['atom_index_x'], df_merge_0_1['atom_index_y']
    return df_merge_0_1

In [5]:
df_train = pd.read_csv(FOLDER + 'train.csv')

In [6]:
df_cos_3j = pd.read_csv(OUTPUT + '20190622_cos_3j.csv')

In [7]:
bond_types = df_train['type'].unique()

for b in bond_types:
    df_train_dist = merge_atom(df_train.query('type == "{}"'.format(b)), df_distance)
    df_train_dist_cos = pd.merge(df_train_dist, df_cos_3j, on='id', how='left')                           
    df_train_dist_cos.to_pickle(OUTPUT + '20190629_dist_bond_dir_cos3j_train_{}.pickle'.format(b))
    del df_train_dist_cos
    gc.collect()